In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = sns.load_dataset('titanic')

In [3]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.shape

(891, 15)

In [7]:
# 查看各列数据类型
df.dtypes

survived          int64
pclass            int64
sex              object
age             float64
sibsp             int64
parch             int64
fare            float64
embarked         object
class          category
who              object
adult_male         bool
deck           category
embark_town      object
alive            object
alone              bool
dtype: object

In [10]:
## 删除不必要的列
drop_list = ["who","adult_male","deck","embark_town","alive","alone","class"]
df.drop(drop_list, axis=1, inplace=True)

In [11]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [17]:
# 查看缺失值
df.isnull().sum().sort_values(ascending=False)

age         177
embarked      2
fare          0
parch         0
sibsp         0
sex           0
pclass        0
survived      0
dtype: int64

In [19]:
# 缺失值处理
df = df[df['embarked'].notnull()]
df['age'].fillna(df['age'].mean(), inplace=True)

In [20]:
# 查看是否还有缺失值
df.isnull().sum().sort_values(ascending=False)

embarked    0
fare        0
parch       0
sibsp       0
age         0
sex         0
pclass      0
survived    0
dtype: int64

In [25]:
# age离散化
df['age'] = pd.cut(df['age'], [0, 18, 90])
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,"(18, 90]",1,0,7.2500,S
1,1,1,female,"(18, 90]",1,0,71.2833,C
2,1,3,female,"(18, 90]",0,0,7.9250,S
3,1,1,female,"(18, 90]",1,0,53.1000,S
4,0,3,male,"(18, 90]",0,0,8.0500,S


In [29]:
## 使用透视表
# 查看不同性别的存活率
table = pd.pivot_table(df, index=['sex'], values='survived')

'''
df是要传入的数据；
index是 values to group by in the rows, 也就是透视表建立时依据哪些字段分组；
values是指对哪些字段进行聚合操作，默认聚合函数是 mean
'''
table

,survived
sex,
female,0.740385
male,0.188908


In [31]:
## 添加列索引
# 添加一个列级分组索引，pclass-客票级别，共有1，2，3三个级别，1最高
table = pd.pivot_table(df, index=['sex'], columns=['pclass'], values='survived')
table

pclass,1,2,3
sex,,,
female,0.967391,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [32]:
## 多级行索引
# 添加一个行级分组索引， pclass-客票级别
table = pd.pivot_table(df, index=['sex', 'pclass'], values='survived')
table

survived
sex    pclass          
female 1       0.967391
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

In [33]:
## 多级列索引
# 构造两层列级索引：’pclass' 和 'age'
table = pd.pivot_table(df, index='sex', columns=['pclass', 'age'], values='survived')
table

pclass         1                 2                   3          
age      (0, 18]  (18, 90] (0, 18]  (18, 90]   (0, 18]  (18, 90]
sex                                                             
female  0.909091  0.975309     1.0  0.903226  0.511628  0.495050
male    0.800000  0.350427     0.6  0.086022  0.215686  0.121622

In [35]:
## 添加多个聚合列
# 按客票级别分组，每组对两个列进行聚合，是否存活 和 船票价
# 若不指定values, 将对除 index 之外的所有列聚合
table = pd.pivot_table(df, index=['pclass'], values=['survived', 'fare'])
table

,fare,survived
pclass,,
1,84.193516,0.626168
2,20.662183,0.472826
3,13.675550,0.242363


In [42]:
## 自定义聚合函数
table = pd.pivot_table(df, index=["pclass"], values=["survived", "fare"], aggfunc=['mean', sum, 'count'])
table

mean                   sum          count         
             fare  survived        fare survived  fare survived
pclass                                                         
1       84.193516  0.626168  18017.4125      134   214      214
2       20.662183  0.472826   3801.8417       87   184      184
3       13.675550  0.242363   6714.6951      119   491      491

In [44]:
## 添加汇总项
# 按行、列进行汇总，指定汇总列名为 total 默认为 all
table = pd.pivot_table(df, index='sex', columns='pclass', values='survived', aggfunc='count', margins=True, margins_name='Total')
table

pclass,1,2,3,Total
sex,,,,
female,92,76,144,312
male,122,108,347,577
Total,214,184,491,889
